# Librerias

In [ ]:
!pip install biopython 
!pip install pysam

import pandas as pd
import pysam
import numpy as np
import Bio
import collections
import json
from Bio import SeqIO

# Objetivo 1

In [ ]:
# Prinicipio objetivo 3 (prueba con 2 secuencias cortas)

def comparar_secuencias (secuencia1,secuencia2): 
    lista = []
    lista2 = []
    identidades = ''
    

    lista_secuencias = [secuencia1,secuencia2]
    for posicion1 in range(len(secuencia1)):
        for posicion2 in range(len(secuencia2)):
            if secuencia1[posicion1] != secuencia2[posicion2]:
                continue
                
            elif secuencia1[posicion1] == secuencia2[posicion2]:

                posicion01 = posicion1
                posicion02 = posicion2
                
                while secuencia1[posicion01] == secuencia2[posicion02]:
                    identidades = identidades + secuencia1[posicion01]
                    posicion01 = posicion01 + 1
                    posicion02 = posicion02 + 1
                    if posicion02 >= len(secuencia2) or posicion01 >= len(secuencia1):
                        break
                if len(identidades) > 3:
                    lista.append(identidades)

                identidades = ''
                
    # se buscan las coinicidencias perfectas (todas las posibles) entre ambas secuencias, y se agregan a una lista.

    for secuencia in lista:
        for secuencia2 in lista_secuencias:
            if secuencia2.startswith(secuencia):
                lista2.append(secuencia)
    # dentro de la lista de coincidencias perfectas se buscan aquellas que se encuentran al inicio de alguna de las 
    # secuencias y se agregan a otra lista.
    secuencia_final = []
    try:
        secuencia_consenso = max(lista2, key=len)
    #print(secuencia_consenso)
    except:
        secuencia_consenso = ('')

    # se selecciona la coinicidencia más larga de todas, de las que se encuentran al inicio de alguna de las secuencias. 

    for secuencia2 in lista_secuencias:
        #print(secuencia2)
        if secuencia2.endswith(secuencia_consenso):
            for secuencia3 in lista_secuencias:
                if secuencia3.startswith(secuencia_consenso):
                    secuencia_final = secuencia2 +secuencia3[len(secuencia_consenso):]
    return(secuencia_consenso)


seq1 = 'GTCTTGTGCGCATTGCGGGCGGGATACGTCTCTGGGCGCTCGGGGAAACT'
seq2 = 'CTCTGGGCGCTCGGGGAAACTAATTCCGGGGAAACCGGGGAAACCTTAAC'
seq3 = 'GGGAAACCGGGGAAACCTTAACTCGACTTAACCCTACACTCGGTGACACT'
# Ejemplos de secuencias múltiples con solapamiento para probar
lista_secuencias = (seq1,seq2,seq3)
secuencias_consenso1 = []
secuencias_consenso = []


for secuencia1 in lista_secuencias:
    for secuencia2 in lista_secuencias:
        if secuencia1 == secuencia2:
            continue
        secuencias_consenso1.append(comparar_secuencias(secuencia1,secuencia2)) 
    secuencias_consenso.append(max(secuencias_consenso1, key=len))
secuencias_consenso = list(dict.fromkeys(secuencias_consenso))
# Me quedo con las secuencias consenso sin repetir y las guardo en una lista
contig = ''
comienzo = False
for secuencia in lista_secuencias:
    if len(contig) == 0:
        for secuencia_consenso in secuencias_consenso:
            if secuencia.startswith(secuencia_consenso):
                comienzo = True
        if comienzo == False:
            contig = contig + secuencia
    # Si el contig no tiene contenido se busca la secuencia que su comienzo no coincide con ninguna de las secuencias consenso
    # para empezar
    if len(contig) != 0:
        for secuencia_consenso in secuencias_consenso:
            if contig.endswith(secuencia_consenso):
                for secuencia in lista_secuencias:
                    if secuencia.startswith(secuencia_consenso):
                        contig = contig + secuencia[len(secuencia_consenso):]
                
print(seq1)
print(seq2)
print(seq3)
print(contig)           

GTCTTGTGCGCATTGCGGGCGGGATACGTCTCTGGGCGCTCGGGGAAACT
CTCTGGGCGCTCGGGGAAACTAATTCCGGGGAAACCGGGGAAACCTTAAC
GGGAAACCGGGGAAACCTTAACTCGACTTAACCCTACACTCGGTGACACT
GTCTTGTGCGCATTGCGGGCGGGATACGTCTCTGGGCGCTCGGGGAAACTAATTCCGGGGAAACCGGGGAAACCTTAACTCGACTTAACCCTACACTCGGTGACACT


In [ ]:
# Objetivo 1, con tolerancia a mismatches
def comparar_secuencias (secuencia1,secuencia2,mismatches_permitidos): 
    lista_sec1 = []
    lista_sec2 = []
    lista2 = []
    identidades_sec1 = ''
    identidades_sec2 = ''
    

    lista_secuencias = [secuencia1,secuencia2]
    for posicion1 in range(len(secuencia1)):
        for posicion2 in range(len(secuencia2)):
            if secuencia1[posicion1] != secuencia2[posicion2]:
                continue
                
            elif secuencia1[posicion1] == secuencia2[posicion2]:

                posicion01 = posicion1
                posicion02 = posicion2
                mismatches = 0
                while mismatches <= mismatches_permitidos:
                    identidades_sec1 = identidades_sec1 + secuencia1[posicion01]
                    identidades_sec2 = identidades_sec2 + secuencia2[posicion02]
                    if secuencia1[posicion01] != secuencia2[posicion02]:
                        mismatches = mismatches + 1
                    posicion01 = posicion01 + 1
                    posicion02 = posicion02 + 1
                    if posicion02 >= len(secuencia2) or posicion01 >= len(secuencia1):
                        lista_sec1.append(identidades_sec1)
                        lista_sec2.append(identidades_sec2)
                        break
                identidades_sec1 = ''
                identidades_sec2 = ''
                
    # se buscan las coinicidencias con mismatches hasta la cantidad permitida (todas las posibles) entre ambas secuencias, y se agregan a una lista
    # aquellas que se encuentran al final de alguna de las secuencias que se comparan.

    secuencias_posibles_sec1 = []
    secuencias_posibles_sec2 = []
    
    for secuencia_lista1 in lista_sec1:
        if secuencia1.endswith(secuencia_lista1):
            secuencias_posibles_sec1.append(secuencia_lista1)
    for secuencia_lista2 in lista_sec2:
        if secuencia2.startswith(secuencia_lista2):
            secuencias_posibles_sec2.append(secuencia_lista2)
        
    try:
        secuencia_consenso1 = max(secuencias_posibles_sec1, key=len)
    #print(secuencia_consenso)
    except:
        secuencia_consenso1 = ('')
    try:
        secuencia_consenso2 = max(secuencias_posibles_sec2, key=len)
    #print(secuencia_consenso)
    except:
        secuencia_consenso2 = ('')
    # se agregan a listas tanto el inicio de la secuencia como el final de la secuencia anterior (las bases que coinciden contemplando los mismatches permitidos) 

    
    return(secuencia_consenso1,secuencia_consenso2)
  

In [ ]:
lista_secuencias = []
for record in SeqIO.parse("reads4contig.fastq", "fastq"):
    lista_secuencias.append(str(record.seq))
lista_secuencias = list(dict.fromkeys(lista_secuencias))
secuencias_usadas = []
secuencias_consenso1 = []
secuencias_consenso = []

mismatches = 5
for secuencia1 in lista_secuencias:
    for secuencia2 in lista_secuencias:
        if secuencia1 == secuencia2:
            continue
        secuencias_consenso1.append(comparar_secuencias(secuencia1,secuencia2,mismatches)[0])
        secuencias_consenso1.append(comparar_secuencias(secuencia1,secuencia2,mismatches)[1])
    secuencias_consenso2 = sorted(secuencias_consenso1, key=len,reverse = True)[:2]
    
    secuencias_consenso1 = []
    if len(secuencias_consenso2[0]) == len(secuencias_consenso2[1]):
        secuencias_consenso.append(secuencias_consenso2)
secuencias_consenso   
# Se agregan de a pares las secuencias consenso entre 2 reads (uno al final de la primera secuencia y otro al principio de la segunda)
# esto se hace por el tema de que hay mismatches (viene de la función de la celda anterior que guarde de esta manera las secuencias).

[['GTCTCAGGAAGGAAGTGCCTATTCATTATTACTTTTAAATGCAGAAATCTTAGTGCACACTCCTCACTGTAATGAACAGATTTTGA',
  'GTCTCAGGAAGGAAGTGCCTATTCATTATTACTTTTAAATGCAGAAATCTTAGTGCACACTCCTCACTGTAATGAACAGATTTTGA'],
 ['AGACACACTCACC', 'AGACACACTCACC'],
 ['TCTCAGGAAGGAAGTGCCTATTCATTATTACTTTTAAATGCAGAAATCTTAGTGCACACTCCTCACTGTAATGAACAGATTTTGACGTTCTCCTTCCCTT',
  'TCTCAGGAAGGAAGTGCCTATTCATTATTACTTTTAAATGCAGAAATCTTAGTGCACACTCCTCACTGTAATGAACAGATTTTGACGTTCTCCTTCCCTT'],
 ['AGAACGTCAAAATCTGTTCATTACAGTGAGGAGTGTGCAATAAGATTTCTGCATTTAAAAGTAATAATGAATAGGCACTTTCTTCCTGAGAC',
  'AGAACGTCAAAATCTGTTCATTACAGTGAGGAGTGTGCACTAAGATTTCTGCATTTAAAAGTAATAATGAATAGGCACTTCCTTCCTGAGAC'],
 ['AGGAAGGAAGTGCCTATTCATTATTACTTTTAAATGCAGAAATCTTAGTGCACACTCCTCACTGTAATGAACAGATTTTGACGTTCTCCTTCCCTTT',
  'AGGAAGGAAGTGCCTATTCATTATTACTTTTAAATGCAGAAATCTTAGTGCACACTCCTCACTGTAATGAACAGATTTTGACGTTCTCCTTCCCTTT'],
 ['ATTCATTATTACTTTTAAATGCAGAAATCTTAGTGCACACTCCTCACTGTAATGAACAGATTTTGACGTTCTCCTTCCCTTTTTTA',
  'ATTCATTATTACTTTTAAATGCAGAAATCTTAGTGCACACTCCTCACTGTAATGAACAGATTTTGA

In [ ]:
contig = ''
comienzo = False
for secuencia in lista_secuencias:
    if len(contig) == 0:
        for par_secuencias_consenso in secuencias_consenso:
            for secuencia_consenso in par_secuencias_consenso:
                if secuencia.startswith(secuencia_consenso):
                    comienzo = True
        if comienzo == False:
            contig = contig + secuencia
    # Si el contig no tiene contenido se busca la secuencia que su comienzo no coincide con ninguna de las secuencias consenso
    # para empezar
    if len(contig) != 0:
        for par_secuencias_consenso in secuencias_consenso:
            if contig.endswith(par_secuencias_consenso[0]):
                for secuencia in lista_secuencias:
                    if secuencia.startswith(par_secuencias_consenso[1]):
                        contig = contig + secuencia[len(par_secuencias_consenso[0]):]
            elif contig.endswith(par_secuencias_consenso[1]):
                for secuencia in lista_secuencias:
                    if secuencia.startswith(par_secuencias_consenso[0]):
                        contig = contig + secuencia[len(par_secuencias_consenso[0]):]
    # Se va armando el contig mirando los pares de secuencias consenso. Si el final de la secuencia coincide con una de las 2 secuencias del par, se agrega al contig
    # la secuencia que empieza con la otra secuencia consenso del par.                    
print(contig)   
print(len(contig))
# El contig me dio más corto de lo que esperaba (239 bases), pero viendo que hay muchísimas repeticiones dentro de los reads del archivo, pienso que puede ser factible
# cada read tiene 101 bases.

TGAGCAGGTGAGTGTGTCTCAGGAAGGAAGTGCCTATTCATTATTACTTTTAAATGCAGAAATCTTAGTGCACACTCCTCACTGTAATGAACAGATTTTGACGTTCTCCTTCCCTTTTTTACATTTGTAAAGTGCTCTGCAAAACTAAACCAAAAGCAGTTCAAATGAATACATAGATGTAACAATCAATGACCTTGACCCTGCCAGTACCAAGAGAGTTAAGTACAAGTGCTCCTCTC
239


# Objetivo 2

In [ ]:
samfile = pysam.AlignmentFile("./aln.bam","rb")
data = {}
coverage = []
for pileupcolumn in samfile.pileup("NC_002745.2", 19500,19550):
    #print ("\ncoverage at base %s = %s" %  (pileupcolumn.pos, pileupcolumn.n))
    coverage.append(pileupcolumn.n)
    bases = []
    for pileupread in pileupcolumn.pileups:
        if not pileupread.is_del and not pileupread.is_refskip:
            base = pileupread.alignment.query_sequence[pileupread.query_position]
            bases.append(base)
    data[pileupcolumn.pos]  =  collections.Counter(bases)   
with open("bases.json","w") as h:
    json.dump(data,h)
print("la maxima cobertura esta en la posicion %s y es de %s" % (coverage.index(max(coverage)),max(coverage)))
print("la mimnima cobertura esta en la posicion %s y es de %s" %  (coverage.index(min(coverage)),min(coverage)))

la maxima cobertura esta en la posicion 87 y es de 531
la mimnima cobertura esta en la posicion 209 y es de 7


In [ ]:
data

In [ ]:
#2b

data = json.load(open("bases.json"))
umbral = 0.01
for record in SeqIO.parse("genomic.fasta", "fasta"):
  ref = record
variantes = []
for key in data.keys():
    for var in data[key]:
      if data[key][var]/sum([data[key][A] for A in data[key]]) >= umbral:
        if var != ref[int(key)]:
          variantes.append((key, ref[int(key)], var, data[key][var]/sum([data[key][A] for A in data[key]])))

In [ ]:
variantes

[('19426', 'A', 'G', 0.02127659574468085),
 ('19427', 'A', 'G', 0.0425531914893617),
 ('19500', 'T', 'C', 1.0),
 ('19619', 'A', 'T', 0.015625),
 ('19627', 'C', 'G', 0.05)]

In [ ]:
"""Ejercicio 2c: 
Agregue al código anterior un filtro que le permita definir en función de dos parámetros: 
i) cantidad de lecturas (o sea profundidad de) que posee la variante y
ii) cantidad de lecturas (o sea profundidad) de la posición 
Cuales de las variantes potenciales pasan (o no) el filtro, y luego escríbalas en formato VCF
iii) Utilizar la info extra de calidad para elegir si la variante es real o no
print(pileupcolumn.get_query_qualities())
print(pileupcolumn.get_mapping_qualities())"""
samfile = pysam.AlignmentFile("./aln.bam","rb")
data = {}
data_exp = {}
coverage = []
for pileupcolumn in samfile.pileup("NC_002745.2", 19500,19550):
    #print ("\ncoverage at base %s = %s" %  (pileupcolumn.pos, pileupcolumn.n))
    coverage.append(pileupcolumn.n)
    bases = []
    for pileupread in pileupcolumn.pileups:
        if not pileupread.is_del and not pileupread.is_refskip:
            base = pileupread.alignment.query_sequence[pileupread.query_position]
            bases.append(base)
    data[pileupcolumn.pos]  =  collections.Counter(bases) 
    data_exp[pileupcolumn.pos]  = [bases, pileupcolumn.get_query_qualities()]
    #print(pileupcolumn.get_query_qualities())
    #print(pileupcolumn.get_mapping_qualities())


In [ ]:
umbral_porcentaje = 0.01
umbral_lecturas = 40 #si pongo valores mas loficos pierdo todas las variantes menos una, y como ejemplo esta bueno que de mas de uno..
umbral_calidad = 35
for record in SeqIO.parse("genomic.fasta", "fasta"):
    ref = record
variantes = []
variantes_exp = []
for key in data.keys():
    for var in data[key]:
      if data[key][var]/sum([data[key][A] for A in data[key]]) >= umbral_porcentaje and len(data_exp[int(key)][0]) >= umbral_lecturas:
        if var != ref[int(key)]:
          calidades = []
          for i, lectura in enumerate(data_exp[int(key)][0]):
            if lectura == var:
              calidades.append(data_exp[int(key)][1][i])
          if sum(calidades)/len(calidades) >= umbral_calidad:

            variantes.append((key, ref[int(key)], var, data[key][var]/sum([data[key][A] for A in data[key]]), len(data_exp[int(key)][0]), sum(calidades)/len(calidades)))
            variantes_exp.append((key, data_exp[int(key)][0],data_exp[int(key)][1]))
variantes = pd.DataFrame (variantes, columns = ["Posicion", "referencia", "variante", "porcentaje", "profundidad", "calidad promedio"])
print(variantes)



   Posicion referencia variante  porcentaje  profundidad  calidad promedio
0     19426          A        G    0.021277           47         37.000000
1     19427          A        G    0.042553           47         37.000000
2     19500          T        C    1.000000          516         37.046512
3     19619          A        T    0.015625           64         37.000000


In [ ]:
#3

In [ ]:
import pandas as pd
df = pd.read_table("variants.ann.vcf",sep="\t",skiprows=42) 
filtro = "missense_variant"
df_miss = df[df["INFO"].str.contains(filtro)]
df_miss

"""
for i in range(len(df["INFO"])):
  if df["INFO"][i] != "":
    try:
      df["INFO"][i] = df["INFO"][i].split(";")
    except:
      pass
#misssense
largo = len(df["INFO"])
print(largo)
miss = [True if ("missense_variant" in df["INFO"][i][-1]) else False for i in range(200)]
"""

'\nfor i in range(len(df["INFO"])):\n  if df["INFO"][i] != "":\n    try:\n      df["INFO"][i] = df["INFO"][i].split(";")\n    except:\n      pass\n#misssense\nlargo = len(df["INFO"])\nprint(largo)\nmiss = [True if ("missense_variant" in df["INFO"][i][-1]) else False for i in range(200)]\n'

In [ ]:
for i in df["INFO"].str.contains(filtro):
  print(i)

In [ ]:
# Objetivo 3- Otra versión
import pandas as pd
df = pd.read_table("./variants.ann.vcf",sep="\t",skiprows=42) 

tipo_cambio = []
for cambio in df['INFO']:
    tipo_cambio.append(cambio.split('|')[1])

#lista_cambios = []
#for info in df['INFO']:
#    if len(info.split('|')[10]) == 0:
#        lista_cambios.append('Sin información')
#    else: 
#        lista_cambios.append(info.split('|')[10].split('.')[1])

datos_variantes = df.copy()
#datos_variantes['Modificación en la secuencia de aminoácidos'] = lista_cambios
datos_variantes['Tipo de modificación'] = tipo_cambio

datos_variantes.loc[datos_variantes['Tipo de modificación']=='missense_variant']



,#CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,FORMAT,1300,Tipo de modificación
1,NC_002745.2,4740,.,A,G,14367.03,.,AC=2;AF=1.00;AN=2;DP=399;ExcessHet=3.0103;FS=0...,GT:AD:DP:GQ:PL,"1/1:0,399:399:99:14381,1196,0",missense_variant
3,NC_002745.2,16204,.,C,T,12128.03,.,AC=2;AF=1.00;AN=2;DP=362;ExcessHet=3.0103;FS=0...,GT:AD:DP:GQ:PL,"1/1:0,362:362:99:12142,1084,0",missense_variant
4,NC_002745.2,19501,.,T,C,15485.03,.,AC=2;AF=1.00;AN=2;DP=447;ExcessHet=3.0103;FS=0...,GT:AD:DP:GQ:PL,"1/1:0,447:447:99:15499,1341,0",missense_variant
9,NC_002745.2,36654,.,T,G,60.60,.,AC=1;AF=0.500;AN=2;DP=4;ExcessHet=3.0103;FS=0....,GT:AD:DP:GQ:PGT:PID:PL:PS,"0|1:4,0:4:68:0|1:36650_C_T:68,0,162:36650",missense_variant
10,NC_002745.2,36655,.,T,G,60.60,.,AC=1;AF=0.500;AN=2;BaseQRankSum=0.431;DP=6;Exc...,GT:AD:DP:GQ:PGT:PID:PL:PS,"0|1:4,2:6:68:0|1:36650_C_T:68,0,162:36650",missense_variant
...,...,...,...,...,...,...,...,...,...,...,...
2196,NC_002745.2,2753061,.,T,C,9568.03,.,AC=2;AF=1.00;AN=2;DP=266;ExcessHet=3.0103;FS=0...,GT:AD:DP:GQ:PL,"1/1:0,266:266:99:9582,798,0",missense_variant
2197,NC_002745.2,2754826,.,G,A,6688.03,.,AC=2;AF=1.00;AN=2;DP=192;ExcessHet=3.0103;FS=0...,GT:AD:DP:GQ:PL,"1/1:0,192:192:99:6702,576,0",missense_variant
2198,NC_002745.2,2754912,.,A,C,11513.03,.,AC=2;AF=1.00;AN=2;DP=340;ExcessHet=3.0103;FS=0...,GT:AD:DP:GQ:PL,"1/1:0,340:340:99:11527,1021,0",missense_variant
2203,NC_002745.2,2778556,.,G,A,17905.03,.,AC=2;AF=1.00;AN=2;DP=491;ExcessHet=3.0103;FS=0...,GT:AD:DP:GQ:PL,"1/1:0,491:491:99:17919,1475,0",missense_variant


In [ ]:
datos_variantes.loc[datos_variantes['Tipo de modificación'].isin(['synonymous_variant','frameshift_variant'])]

,#CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,FORMAT,1300,Tipo de modificación
7,NC_002745.2,34822,.,TG,T,12119.03,.,AC=2;AF=1.00;AN=2;DP=300;ExcessHet=3.0103;FS=0...,GT:AD:DP:GQ:PL,"1/1:0,300:300:99:12133,903,0",frameshift_variant
12,NC_002745.2,36675,.,T,C,142.60,.,AC=1;AF=0.500;AN=2;BaseQRankSum=0.703;DP=8;Exc...,GT:AD:DP:GQ:PGT:PID:PL:PS,"0|1:5,3:8:99:0|1:36667_G_T:150,0,240:36667",synonymous_variant
13,NC_002745.2,36681,.,A,G,145.60,.,AC=1;AF=0.500;AN=2;BaseQRankSum=-6.000e-02;DP=...,GT:AD:DP:GQ:PGT:PID:PL:PS,"0|1:5,4:9:99:0|1:36667_G_T:153,0,198:36667",synonymous_variant
15,NC_002745.2,36690,.,T,C,142.60,.,AC=1;AF=0.500;AN=2;BaseQRankSum=1.20;DP=10;Exc...,GT:AD:DP:GQ:PGT:PID:PL:PS,"0|1:6,4:10:99:0|1:36667_G_T:150,0,222:36667",synonymous_variant
16,NC_002745.2,36705,.,T,G,271.60,.,AC=1;AF=0.500;AN=2;BaseQRankSum=-1.550e+00;DP=...,GT:AD:DP:GQ:PGT:PID:PL:PS,"0|1:5,7:12:99:0|1:36705_T_G:279,0,189:36705",synonymous_variant
...,...,...,...,...,...,...,...,...,...,...,...
2200,NC_002745.2,2755727,.,A,G,14059.03,.,AC=2;AF=1.00;AN=2;DP=375;ExcessHet=3.0103;FS=0...,GT:AD:DP:GQ:PL,"1/1:0,375:375:99:14073,1126,0",synonymous_variant
2201,NC_002745.2,2768496,.,A,G,12483.03,.,AC=2;AF=1.00;AN=2;DP=359;ExcessHet=3.0103;FS=0...,GT:AD:DP:GQ:PGT:PID:PL:PS,"1|1:0,359:359:99:1|1:2768496_A_G:12497,1078,0:...",synonymous_variant
2207,NC_002745.2,2787033,.,T,G,10056.03,.,AC=2;AF=1.00;AN=2;DP=308;ExcessHet=3.0103;FS=0...,GT:AD:DP:GQ:PL,"1/1:0,308:308:99:10070,923,0",synonymous_variant
2208,NC_002745.2,2797855,.,G,A,12181.03,.,AC=2;AF=1.00;AN=2;DP=373;ExcessHet=3.0103;FS=0...,GT:AD:DP:GQ:PL,"1/1:0,373:373:99:12195,1118,0",synonymous_variant


In [ ]:
#Ejercicio 3

df = pd.read_table("./variants.ann.vcf",sep="\t",skiprows=42)
#print(df.INFO)

filtro = df[df['INFO'].str.contains("missense_variant", case=False)]
print("Estas son las variantes missense:")
print(filtro)

In [ ]:
#Ejercicio 3 
df = pd.read_table("./variants.ann.vcf",sep="\t",skiprows=42)
#print(df.INFO)

filtro = df[df['INFO'].str.contains("frameshift_variant", case=False)]
print("Estas son las variantes de cambio de marco de lectura:")
print(filtro)


filtro2 = df[df['INFO'].str.contains("synonymous_variant", case=False)]
print("Estas son las variantes sinonimas:")
print(filtro2)

Estas son las variantes de cambio de marco de lectura:
           #CHROM  ...                                               1300
7     NC_002745.2  ...                       1/1:0,300:300:99:12133,903,0
55    NC_002745.2  ...    1|1:0,45:45:99:1|1:36986_CGT_C:2025,135,0:36986
58    NC_002745.2  ...    1|1:0,50:50:99:1|1:36986_CGT_C:2250,151,0:36986
64    NC_002745.2  ...     0|1:12,2:14:48:0|1:42332_CGT_C:48,0,1064:42332
67    NC_002745.2  ...     0|1:13,3:16:87:0|1:42332_CGT_C:87,0,1277:42332
...           ...  ...                                                ...
2134  NC_002745.2  ...  0|1:229,49:278:99:0|1:2550270_G_A:1429,0,47256...
2135  NC_002745.2  ...  0|1:229,25:254:99:0|1:2550270_G_A:1381,0,47329...
2136  NC_002745.2  ...  0|1:232,22:254:99:0|1:2550270_G_A:348,0,47414:...
2153  NC_002745.2  ...                        1/1:1,190:191:99:5768,551,0
2173  NC_002745.2  ...                        1/1:0,179:179:99:5634,538,0

[104 rows x 10 columns]
Estas son las variantes sinonima